## Imports:

In [1]:
# from __future__ import unicode_literals
from __future__ import division
from __future__ import print_function

import os

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 60)
# import tables as h5

import itertools as it
from collections import defaultdict

import numpy as np
import pandas as pd

xls_writer = pd.ExcelWriter


import scipy
import statsmodels.api as sm
import statsmodels.stats.multitest as smm

import munch

from spartan.utils.genome_specific.GfusI1 import GfusI1_0
from spartan.utils.fastas import ParseFastA


## File paths:

In [2]:
# define paths to files

in_file = "/home/gus/MEGAsync/zim/main/Yale/Collaborations/Hongyu-tsetse/Yao/snp_list_afterFS_with_p_value_LD_1"
out_file = "/home/gus/MEGAsync/zim/main/Yale/Collaborations/Hongyu-tsetse/Yao/snp_list_afterFS_with_p_value_LD_1_converted.csv"

out_bed = "{p}.bed".format(p=os.path.splitext(out_file)[0])

contig_name_length_path = "/home/gus/Dropbox/uganda_data/data_repos/genome_info/assembly_info/contig_name_length.csv"


In [3]:
# load contig info file
contig_info = pd.read_csv(contig_name_length_path)
contig_info.head()

,kk_name,scaf_name,length
0,KK352346.1,Scaffold566,193315
1,KK352610.1,Scaffold839,83110
2,KK352241.1,Scaffold458,243873
3,JFJR01012964.1,JFJR01012964.1,3083
4,KK352052.1,Scaffold268,427914


In [4]:
def get_contig_name_map(df):
    return {key:value for key,value in zip(contig_info.kk_name.values,contig_info.scaf_name.values)}

In [5]:
contig_name_map = get_contig_name_map(contig_info)
contig_name_map["KK351785.1"]

'Scaffold0'

In [6]:
contig_name_map["JFJR01006593.1"]

'JFJR01006593.1'

------------------

#  Convert `snp_list_afterFS_with_p_value_LD_1`

In [7]:
!head $in_file

In [8]:
def add_chrom_loc(df, name_map):
    df["CHROM"] = df.SNP.apply(lambda x: name_map[x.split(':')[1]])
    df["LOC"] = df.SNP.apply(lambda x: int(x.split(':')[2]))
    
    

In [9]:
snp_list_afterFS_with_p_value_LD_1 = pd.read_table(in_file)
add_chrom_loc(snp_list_afterFS_with_p_value_LD_1,contig_name_map)
snp_list_afterFS_with_p_value_LD_1.head()

,SNP,P,CHROM,LOC
1,GFvariants_VB2014a_tvcf:KK352172.1:259728,0.00459,Scaffold389,259728
2,GFvariants_VB2014d_tvcf:KK351922.1:615407,0.00467,Scaffold137,615407
3,GFvariants_VB2014c_tvcf:JFJR01012925.1:2545,0.00488,JFJR01012925.1,2545
4,GFvariants_VB2014e_tvcf:KK351818.1:1387926,0.00630,Scaffold33,1387926
5,GFvariants_VB2014d_tvcf:KK352538.1:42842,0.01270,Scaffold761,42842


In [10]:
snp_list_afterFS_with_p_value_LD_1.to_csv(out_file)

# Build and write the corresponding BED file

In [11]:
bed_table = snp_list_afterFS_with_p_value_LD_1[["CHROM","LOC"]].copy()
bed_table = bed_table.rename(columns={"LOC":"END"}).copy()
bed_table['START'] = bed_table.END - 1
bed_table = bed_table[["CHROM","START","END"]].copy()
bed_table.head()

,CHROM,START,END
1,Scaffold389,259727,259728
2,Scaffold137,615406,615407
3,JFJR01012925.1,2544,2545
4,Scaffold33,1387925,1387926
5,Scaffold761,42841,42842


In [12]:
bed_table.to_csv(out_bed, header=False, index=False, sep='\t')

In [13]:
!head $out_bed

Scaffold389	259727	259728
Scaffold137	615406	615407
JFJR01012925.1	2544	2545
Scaffold33	1387925	1387926
Scaffold761	42841	42842
Scaffold175	347131	347132
Scaffold8	1744225	1744226
Scaffold993	1107	1108
Scaffold48	1013372	1013373
Scaffold104	688652	688653
